In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2000 to 2020 Exports\MODIS LC  (2006 to 2015 but consistent)\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Difference"
output_dir = r"C:\Users\llccl\Downloads\1.Difference"

os.makedirs(output_dir, exist_ok=True)

# Iterate over each LST image
for lst_filename in os.listdir(lst_dir):
    if lst_filename.endswith(".tif"):
        lst_path = os.path.join(lst_dir, lst_filename)
        output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_LC_Type1_0615.csv")

        transition_matrix = np.full((10, 10), np.nan)

        # Open the LST raster
        with rasterio.open(lst_path) as lst_ds:
            lst_data = lst_ds.read(1)
            lst_transform = lst_ds.transform
            lst_crs = lst_ds.crs

        # Iterate over masks
        for filename in os.listdir(mask_dir):
            if filename.endswith("_Mask_mosaic.tif"):
                parts = filename.split('_')
                class_from = int(parts[0].replace("Class", "")) - 1
                class_to = int(parts[2].replace("Class", "")) - 1

                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    # Reproject mask to LST CRS
                    if mask_ds.crs != lst_crs:
                        transform, width, height = calculate_default_transform(
                            mask_ds.crs, lst_crs, mask_ds.width, mask_ds.height, *mask_ds.bounds
                        )
                        reprojected_mask = np.zeros((height, width), dtype=np.uint8)
                        reproject(
                            source=rasterio.band(mask_ds, 1),
                            destination=reprojected_mask,
                            src_transform=mask_ds.transform,
                            src_crs=mask_ds.crs,
                            dst_transform=lst_transform,
                            dst_crs=lst_crs,
                            resampling=Resampling.nearest,
                        )
                    else:
                        reprojected_mask = mask_ds.read(1)

                    # Ensure shape compatibility
                    reprojected_mask = reprojected_mask[: lst_data.shape[0], : lst_data.shape[1]]

                    # Extract LST values where mask == 1
                    masked_lst_values = lst_data[reprojected_mask == 1]
                    masked_lst_values = masked_lst_values[~np.isnan(masked_lst_values)]

                    # Apply IQR filtering
                    if masked_lst_values.size > 0:
                        q1 = np.percentile(masked_lst_values, 25)
                        q3 = np.percentile(masked_lst_values, 75)
                        iqr_filtered_values = masked_lst_values[(masked_lst_values >= q1) & (masked_lst_values <= q3)]

                        if iqr_filtered_values.size > 0:
                            mean_value = np.mean(iqr_filtered_values)
                            transition_matrix[class_from, class_to] = mean_value

        # Save transition matrix
        transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{i+1}' for i in range(10)], index=[f'Class{i+1}' for i in range(10)])
        transition_df.to_csv(output_csv, index=True)

        print(f"Saved: {output_csv}")


Saved: C:\Users\llccl\Downloads\1.Difference\MOYD21A1DN_2016-2020_2000-2005_Average_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\1.Difference\MOYD21A1DN_2016-2020_2000-2005_RegressCombination_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\1.Difference\MOYD21A1D_2016-2020_2000-2005_mean_Average_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\1.Difference\MOYD21A1N_2016-2020_2000-2005_mean_Average_LC_Type1_0615.csv


In [2]:
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2000 to 2020 Exports\MODIS LC  (2006 to 2015 but consistent)\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Linear Trend"
output_dir = r"C:\Users\llccl\Downloads\2.LinearTrend"

os.makedirs(output_dir, exist_ok=True)

# Iterate over each LST image
for lst_filename in os.listdir(lst_dir):
    if lst_filename.endswith(".tif"):
        lst_path = os.path.join(lst_dir, lst_filename)
        output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_LC_Type1_0615.csv")

        transition_matrix = np.full((10, 10), np.nan)

        # Open the LST raster
        with rasterio.open(lst_path) as lst_ds:
            lst_data = lst_ds.read(1)
            lst_transform = lst_ds.transform
            lst_crs = lst_ds.crs

        # Iterate over masks
        for filename in os.listdir(mask_dir):
            if filename.endswith("_Mask_mosaic.tif"):
                parts = filename.split('_')
                class_from = int(parts[0].replace("Class", "")) - 1
                class_to = int(parts[2].replace("Class", "")) - 1

                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    # Reproject mask to LST CRS
                    if mask_ds.crs != lst_crs:
                        transform, width, height = calculate_default_transform(
                            mask_ds.crs, lst_crs, mask_ds.width, mask_ds.height, *mask_ds.bounds
                        )
                        reprojected_mask = np.zeros((height, width), dtype=np.uint8)
                        reproject(
                            source=rasterio.band(mask_ds, 1),
                            destination=reprojected_mask,
                            src_transform=mask_ds.transform,
                            src_crs=mask_ds.crs,
                            dst_transform=lst_transform,
                            dst_crs=lst_crs,
                            resampling=Resampling.nearest,
                        )
                    else:
                        reprojected_mask = mask_ds.read(1)

                    # Ensure shape compatibility
                    reprojected_mask = reprojected_mask[: lst_data.shape[0], : lst_data.shape[1]]

                    # Extract LST values where mask == 1
                    masked_lst_values = lst_data[reprojected_mask == 1]
                    masked_lst_values = masked_lst_values[~np.isnan(masked_lst_values)]

                    # Apply IQR filtering
                    if masked_lst_values.size > 0:
                        q1 = np.percentile(masked_lst_values, 25)
                        q3 = np.percentile(masked_lst_values, 75)
                        iqr_filtered_values = masked_lst_values[(masked_lst_values >= q1) & (masked_lst_values <= q3)]

                        if iqr_filtered_values.size > 0:
                            mean_value = np.mean(iqr_filtered_values)
                            transition_matrix[class_from, class_to] = mean_value

        # Save transition matrix
        transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{i+1}' for i in range(10)], index=[f'Class{i+1}' for i in range(10)])
        transition_df.to_csv(output_csv, index=True)

        print(f"Saved: {output_csv}")


Saved: C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1DN_2000-2020_LinearTrend_Average_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1DN_2000-2020_LinearTrend_RegressCombination_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1D_2000-2020_LinearTrend_Average_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1N_2000-2020_LinearTrend_Average_LC_Type1_0615.csv


In [3]:
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2000 to 2020 Exports\MODIS LC  (2006 to 2015 but consistent)\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Mean"
output_dir = r"C:\Users\llccl\Downloads\3.Mean"

os.makedirs(output_dir, exist_ok=True)

# Iterate over each LST image
for lst_filename in os.listdir(lst_dir):
    if lst_filename.endswith(".tif"):
        lst_path = os.path.join(lst_dir, lst_filename)
        output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_LC_Type1_0615.csv")

        transition_matrix = np.full((10, 10), np.nan)

        # Open the LST raster
        with rasterio.open(lst_path) as lst_ds:
            lst_data = lst_ds.read(1)
            lst_transform = lst_ds.transform
            lst_crs = lst_ds.crs

        # Iterate over masks
        for filename in os.listdir(mask_dir):
            if filename.endswith("_Mask_mosaic.tif"):
                parts = filename.split('_')
                class_from = int(parts[0].replace("Class", "")) - 1
                class_to = int(parts[2].replace("Class", "")) - 1

                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    # Reproject mask to LST CRS
                    if mask_ds.crs != lst_crs:
                        transform, width, height = calculate_default_transform(
                            mask_ds.crs, lst_crs, mask_ds.width, mask_ds.height, *mask_ds.bounds
                        )
                        reprojected_mask = np.zeros((height, width), dtype=np.uint8)
                        reproject(
                            source=rasterio.band(mask_ds, 1),
                            destination=reprojected_mask,
                            src_transform=mask_ds.transform,
                            src_crs=mask_ds.crs,
                            dst_transform=lst_transform,
                            dst_crs=lst_crs,
                            resampling=Resampling.nearest,
                        )
                    else:
                        reprojected_mask = mask_ds.read(1)

                    # Ensure shape compatibility
                    reprojected_mask = reprojected_mask[: lst_data.shape[0], : lst_data.shape[1]]

                    # Extract LST values where mask == 1
                    masked_lst_values = lst_data[reprojected_mask == 1]
                    masked_lst_values = masked_lst_values[~np.isnan(masked_lst_values)]

                    # Apply IQR filtering
                    if masked_lst_values.size > 0:
                        q1 = np.percentile(masked_lst_values, 25)
                        q3 = np.percentile(masked_lst_values, 75)
                        iqr_filtered_values = masked_lst_values[(masked_lst_values >= q1) & (masked_lst_values <= q3)]

                        if iqr_filtered_values.size > 0:
                            mean_value = np.mean(iqr_filtered_values)
                            transition_matrix[class_from, class_to] = mean_value

        # Save transition matrix
        transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{i+1}' for i in range(10)], index=[f'Class{i+1}' for i in range(10)])
        transition_df.to_csv(output_csv, index=True)

        print(f"Saved: {output_csv}")


Saved: C:\Users\llccl\Downloads\3.Mean\MOYD21A1DN_2000-2020_mean_Average_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\3.Mean\MOYD21A1DN_2000-2020_mean_RegressCombination_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\3.Mean\MOYD21A1D_2000-2020_mean_Average_LC_Type1_0615.csv
Saved: C:\Users\llccl\Downloads\3.Mean\MOYD21A1N_2000-2020_mean_Average_LC_Type1_0615.csv
